This is a notebook for *short and sweet* examples and links for useful pandas recipes.

Adding interesting links and/or inline examples to this section is a great *First Pull Request*.

Simplified, condensed, new-user friendly, in-line examples have been inserted where possible to
augment the Stack-Overflow and GitHub links.  Many of the links contain expanded information,
above what the in-line examples offer.

pandas (pd) and NumPy (np) are the only two abbreviated imported modules. The rest are kept
explicitly imported for newer users.

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df


### Idioms -  If Then

In [ ]:
# An if-then on one column
df.loc[df.AAA >= 5, "BBB"] = -1
df

#An if-then with assignment to 2 columns:
df.loc[df.AAA >= 5, ["BBB", "CCC"]] = 555
df

#Add another line with different logic, to do the -else
df.loc[df.AAA < 5, ["BBB", "CCC"]] = 2000
df

# Or use pandas where after you've set up a mask
df_mask = pd.DataFrame(
    {"AAA": [True] * 4, "BBB": [False] * 4, "CCC": [True, False] * 2}
)
df.where(df_mask, -1000)

In [ ]:
#if-then-else using NumPy's where()

df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df
df["logic"] = np.where(df["AAA"] > 5, "high", "low")
df

### Splitting

In [ ]:
# Split a frame with a boolean criterion

df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df

df[df.AAA <= 5]
df[df.AAA > 5]

# Building criteria

In [ ]:
# Select with multi-column criteria
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df

# and (without assignment returns a Series)
df.loc[(df["BBB"] < 25) & (df["CCC"] >= -40), "AAA"]


In [ ]:
# or (without assignment returns a Series)
df.loc[(df["BBB"] > 25) | (df["CCC"] >= -40), "AAA"]

# or (with assignment modifies the DataFrame.)
df.loc[(df["BBB"] > 25) | (df["CCC"] >= 75), "AAA"] = 0.1
df


In [ ]:
# Select rows with data closest to certain value using argsort
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df
aValue = 43.0
df.loc[(df.CCC - aValue).abs().argsort()]

# Dynamically reduce a list of criteria using a binary operators
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df

In [ ]:

Crit1 = df.AAA <= 5.5
Crit2 = df.BBB == 10.0
Crit3 = df.CCC > -40.0

# One could hard code:
AllCrit = Crit1 & Crit2 & Crit3

# Or it can be done with a list of dynamically built criteria
import functools
CritList = [Crit1, Crit2, Crit3]
AllCrit = functools.reduce(lambda x, y: x & y, CritList)

df[AllCrit]


### Selection Dataframes

In [ ]:
#Using both row labels and value conditionals
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df

df[(df.AAA <= 6) & (df.index.isin([0, 2, 4]))]

#Use loc for label-oriented slicing and iloc positional slicing :issue:`2904`
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]},
    index=["foo", "bar", "boo", "kar"],
)

In [ ]:
# There are 2 explicit slicing methods, with a third general case

# 1. Positional-oriented (Python slicing style : exclusive of end)
df.iloc[0:3]  # Positional


# 2. Label-oriented (Non-Python slicing style : inclusive of end)
df.loc["bar":"kar"]  # Label


# 3. General (Either slicing style : depends on if the slice contains labels or positions)
df[0:3]
df["bar":"kar"]

# Ambiguity arises when an index consists of integers with a non-zero start or non-unit increment.
data = {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
df2 = pd.DataFrame(data=data, index=[1, 2, 3, 4])  # Note index starts at 1.
df2.iloc[1:3]  # Position-oriented
df2.loc[1:3]  # Label-oriented

In [ ]:

# Using inverse operator (~) to take the complement of a mask
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df

df[~((df.AAA <= 6) & (df.index.isin([0, 2, 4])))]

### New Columns

In [ ]:
# Efficiently and dynamically creating new columns using applymap
df = pd.DataFrame({"AAA": [1, 2, 1, 3], "BBB": [1, 1, 2, 2], "CCC": [2, 1, 3, 1]})
df

source_cols = df.columns  # Or some subset would work too
new_cols = [str(x) + "_cat" for x in source_cols]
categories = {1: "Alpha", 2: "Beta", 3: "Charlie"}

df[new_cols] = df[source_cols].applymap(categories.get)
df

# Keep other columns when using min() with groupby
df = pd.DataFrame(
    {"AAA": [1, 1, 1, 2, 2, 2, 3, 3], "BBB": [2, 1, 3, 4, 5, 1, 2, 3]}
)
df

# Method 1 : idxmin() to get the index of the minimums
df.loc[df.groupby("AAA")["BBB"].idxmin()]

# Method 2 : sort then take first of each
df.sort_values(by="BBB").groupby("AAA", as_index=False).first()



### Multi Indexing

In [ ]:
# Creating a MultiIndex from a labeled frame
df = pd.DataFrame(
    {
        "row": [0, 1, 2],
        "One_X": [1.1, 1.1, 1.1],
        "One_Y": [1.2, 1.2, 1.2],
        "Two_X": [1.11, 1.11, 1.11],
        "Two_Y": [1.22, 1.22, 1.22],
    }
)
df

# As Labelled Index
df = df.set_index("row")
df
# With Hierarchical Columns
df.columns = pd.MultiIndex.from_tuples([tuple(c.split("_")) for c in df.columns])
df
# Now stack & Reset
df = df.stack(0).reset_index(1)
df
# And fix the labels (Notice the label 'level_1' got added automatically)
df.columns = ["Sample", "All_X", "All_Y"]
df


In [ ]:
# Arithmetic - Performing arithmetic with a MultiIndex that needs broadcasting
cols = pd.MultiIndex.from_tuples(
    [(x, y) for x in ["A", "B", "C"] for y in ["O", "I"]]
)
df = pd.DataFrame(np.random.randn(2, 6), index=["n", "m"], columns=cols)
df
df = df.div(df["C"], level=1)
df

In [ ]:
# Slicing - Slicing a MultiIndex with xs
coords = [("AA", "one"), ("AA", "six"), ("BB", "one"), ("BB", "two"), ("BB", "six")]
index = pd.MultiIndex.from_tuples(coords)
df = pd.DataFrame([11, 22, 33, 44, 55], index, ["MyData"])
df

# To take the cross section of the 1st level and 1st axis the index:
# Note : level and axis are optional, and default to zero
df.xs("BB", level=0, axis=0)

# and now the 2nd level of the 1st axis.
df.xs("six", level=1, axis=0)

In [ ]:

# Slicing a MultiIndex with xs, method #2
import itertools

index = list(itertools.product(["Ada", "Quinn", "Violet"], ["Comp", "Math", "Sci"]))
headr = list(itertools.product(["Exams", "Labs"], ["I", "II"]))
indx = pd.MultiIndex.from_tuples(index, names=["Student", "Course"])
cols = pd.MultiIndex.from_tuples(headr)  # Notice these are un-named
data = [[70 + x + y + (x * y) % 3 for x in range(4)] for y in range(9)]
df = pd.DataFrame(data, indx, cols)
df

All = slice(None)
df.loc["Violet"]
df.loc[(All, "Math"), All]
df.loc[(slice("Ada", "Quinn"), "Math"), All]
df.loc[(All, "Math"), ("Exams")]
df.loc[(All, "Math"), (All, "II")]

### Missing data

In [ ]:
# Fill forward a reversed timeseries

df = pd.DataFrame(
    np.random.randn(6, 1),
    index=pd.date_range("2013-08-01", periods=6, freq="B"),
    columns=list("A"),
)
df.loc[df.index[3], "A"] = np.nan
df
df.bfill()

### Grouping

In [ ]:
# Basic grouping with apply
# Unlike agg, apply's callable is passed a sub-DataFrame which gives you access to all the columns

df = pd.DataFrame(
    {
        "animal": "cat dog cat fish dog cat cat".split(),
        "size": list("SSMMMLL"),
        "weight": [8, 10, 11, 1, 20, 12, 12],
        "adult": [False] * 5 + [True] * 2,
    }
)
df

# List the size of the animals with the highest weight.
df.groupby("animal").apply(lambda subf: subf["size"][subf["weight"].idxmax()])

# Using get_group
gb = df.groupby(["animal"])
gb.get_group("cat")

# Apply to different items in a group
def GrowUp(x):
    avg_weight = sum(x[x["size"] == "S"].weight * 1.5)
    avg_weight += sum(x[x["size"] == "M"].weight * 1.25)
    avg_weight += sum(x[x["size"] == "L"].weight)
    avg_weight /= len(x)
    return pd.Series(["L", avg_weight, True], index=["size", "weight", "adult"])


expected_df = gb.apply(GrowUp)
expected_df


In [ ]:

# Expanding apply
S = pd.Series([i / 100.0 for i in range(1, 11)])

def cum_ret(x, y):
    return x * (1 + y)

def red(x):
    return functools.reduce(cum_ret, x, 1.0)

S.expanding().apply(red, raw=True)


# Replacing some values with mean of the rest of a group
df = pd.DataFrame({"A": [1, 1, 2, 2], "B": [1, -1, 1, 2]})
gb = df.groupby("A")

def replace(g):
    mask = g < 0
    return g.where(~mask, g[~mask].mean())

gb.transform(replace)

# Sort groups by aggregated data
df = pd.DataFrame(
    {
        "code": ["foo", "bar", "baz"] * 2,
        "data": [0.16, -0.21, 0.33, 0.45, -0.59, 0.62],
        "flag": [False, True] * 3,
    }
)

code_groups = df.groupby("code")

agg_n_sort_order = code_groups[["data"]].transform(sum).sort_values(by="data")

sorted_df = df.loc[agg_n_sort_order.index]

sorted_df


In [ ]:

# Create multiple aggregated columns
rng = pd.date_range(start="2014-10-07", periods=10, freq="2min")
ts = pd.Series(data=list(range(10)), index=rng)

def MyCust(x):
    if len(x) > 2:
        return x[1] * 1.234
    return pd.NaT

mhc = {"Mean": np.mean, "Max": np.max, "Custom": MyCust}
ts.resample("5min").apply(mhc)
ts

# Create a value counts column and reassign back to the DataFrame
df = pd.DataFrame(
    {"Color": "Red Red Red Blue".split(), "Value": [100, 150, 50, 50]}
)
df
df["Counts"] = df.groupby(["Color"]).transform(len)
df

# Shift groups of the values in a column based on the index
df = pd.DataFrame(
    {"line_race": [10, 10, 8, 10, 10, 8], "beyer": [99, 102, 103, 103, 88, 100]},
    index=[
        "Last Gunfighter",
        "Last Gunfighter",
        "Last Gunfighter",
        "Paynter",
        "Paynter",
        "Paynter",
    ],
)
df
df["beyer_shifted"] = df.groupby(level=0)["beyer"].shift(1)
df

In [ ]:

# Select row with maximum value from each group
df = pd.DataFrame(
    {
        "host": ["other", "other", "that", "this", "this"],
        "service": ["mail", "web", "mail", "mail", "web"],
        "no": [1, 2, 1, 2, 1],
    }
).set_index(["host", "service"])
mask = df.groupby(level=0).agg("idxmax")
df_count = df.loc[mask["no"]].reset_index()
df_count

# Grouping like Python's itertools.groupby
df = pd.DataFrame([0, 1, 0, 1, 1, 1, 0, 1, 1], columns=["A"])
df["A"].groupby((df["A"] != df["A"].shift()).cumsum()).groups
df["A"].groupby((df["A"] != df["A"].shift()).cumsum()).cumsum()


### Splitting

In [ ]:
# Splitting a frame
# Create a list of dataframes, split using a delineation based on logic included in rows.
df = pd.DataFrame(
    data={
        "Case": ["A", "A", "A", "B", "A", "A", "B", "A", "A"],
        "Data": np.random.randn(9),
    }
)

dfs = list(
    zip(
        *df.groupby(
            (1 * (df["Case"] == "B"))
            .cumsum()
            .rolling(window=3, min_periods=1)
            .median()
        )
    )
)[-1]

dfs[0]
dfs[1]
dfs[2]

### Pivot

In [ ]:
# Partial sums and subtotals
df = pd.DataFrame(
    data={
        "Province": ["ON", "QC", "BC", "AL", "AL", "MN", "ON"],
        "City": [
            "Toronto",
            "Montreal",
            "Vancouver",
            "Calgary",
            "Edmonton",
            "Winnipeg",
            "Windsor",
        ],
        "Sales": [13, 6, 16, 8, 4, 3, 1],
    }
)
table = pd.pivot_table(
    df,
    values=["Sales"],
    index=["Province"],
    columns=["City"],
    aggfunc=np.sum,
    margins=True,
)
table.stack("City")

In [ ]:
# Frequency table like plyr in R
grades = [48, 99, 75, 80, 42, 80, 72, 68, 36, 78]
df = pd.DataFrame(
    {
        "ID": ["x%d" % r for r in range(10)],
        "Gender": ["F", "M", "F", "M", "F", "M", "F", "M", "M", "M"],
        "ExamYear": [
            "2007",
            "2007",
            "2007",
            "2008",
            "2008",
            "2008",
            "2008",
            "2009",
            "2009",
            "2009",
        ],
        "Class": [
            "algebra",
            "stats",
            "bio",
            "algebra",
            "algebra",
            "stats",
            "stats",
            "algebra",
            "bio",
            "bio",
        ],
        "Participated": [
            "yes",
            "yes",
            "yes",
            "yes",
            "no",
            "yes",
            "yes",
            "yes",
            "yes",
            "yes",
        ],
        "Passed": ["yes" if x > 50 else "no" for x in grades],
        "Employed": [
            True,
            True,
            True,
            False,
            False,
            False,
            False,
            True,
            True,
            False,
        ],
        "Grade": grades,
    }
)

df.groupby("ExamYear").agg(
    {
        "Participated": lambda x: x.value_counts()["yes"],
        "Passed": lambda x: sum(x == "yes"),
        "Employed": lambda x: sum(x),
        "Grade": lambda x: sum(x) / len(x),
    }
)

In [ ]:
# Plot pandas DataFrame with year over year data
# To create year and month cross tabulation:
df = pd.DataFrame(
    {"value": np.random.randn(36)},
    index=pd.date_range("2011-01-01", freq="M", periods=36),
)

pd.pivot_table(
    df, index=df.index.month, columns=df.index.year, values="value", aggfunc="sum"
)


### Apply

In [ ]:
# Rolling apply to organize - Turning embedded lists into a MultiIndex frame
df = pd.DataFrame(
    data={
        "A": [[2, 4, 8, 16], [100, 200], [10, 20, 30]],
        "B": [["a", "b", "c"], ["jj", "kk"], ["ccc"]],
    },
    index=["I", "II", "III"],
)

def SeriesFromSubList(aList):
    return pd.Series(aList)

df_orgz = pd.concat(
    {ind: row.apply(SeriesFromSubList) for ind, row in df.iterrows()}
)
df_orgz


In [ ]:
# Rolling apply with a DataFrame returning a Series
df = pd.DataFrame(
    data=np.random.randn(2000, 2) / 10000,
    index=pd.date_range("2001-01-01", periods=2000),
    columns=["A", "B"],
)
df

def gm(df, const):
    v = ((((df["A"] + df["B"]) + 1).cumprod()) - 1) * const
    return v.iloc[-1]

s = pd.Series(
    {
        df.index[i]: gm(df.iloc[i: min(i + 51, len(df) - 1)], 5)
        for i in range(len(df) - 50)
    }
)
s


In [ ]:

# Rolling apply with a DataFrame returning a Scalar
rng = pd.date_range(start="2014-01-01", periods=100)
df = pd.DataFrame(
    {
        "Open": np.random.randn(len(rng)),
        "Close": np.random.randn(len(rng)),
        "Volume": np.random.randint(100, 2000, len(rng)),
    },
    index=rng,
)
df

def vwap(bars):
    return (bars.Close * bars.Volume).sum() / bars.Volume.sum()

window = 5
s = pd.concat(
    [
        (pd.Series(vwap(df.iloc[i: i + window]), index=[df.index[i + window]]))
        for i in range(len(df) - window)
    ]
)
s.round(2)

### Merge

In [ ]:
# Concatenate two dataframes with overlapping index (emulate R rbind)
rng = pd.date_range("2000-01-01", periods=6)
df1 = pd.DataFrame(np.random.randn(6, 3), index=rng, columns=["A", "B", "C"])
df2 = df1.copy()

#Depending on df construction, ignore_index may be needed
df = pd.concat([df1, df2], ignore_index=True)
df

# Self Join of a DataFrame
df = pd.DataFrame(
    data={
        "Area": ["A"] * 5 + ["C"] * 2,
        "Bins": [110] * 2 + [160] * 3 + [40] * 2,
        "Test_0": [0, 1, 0, 1, 2, 0, 1],
        "Data": np.random.randn(7),
    }
)
df

df["Test_1"] = df["Test_0"] - 1

pd.merge(
    df,
    df,
    left_on=["Bins", "Area", "Test_0"],
    right_on=["Bins", "Area", "Test_1"],
    suffixes=("_L", "_R"),
)
